<a href="https://colab.research.google.com/github/humbletechy/258_Assignment_2/blob/main/Sim_Based_Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install tqdm
import pandas as pd
from collections import defaultdict
from tqdm import tqdm 
from sklearn.metrics import mean_squared_error
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
df_reviews = pd.read_csv('/content/drive/MyDrive/CSE258/CA_reviews.csv')  

In [4]:
df_reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,ZKvDG2sBvHVdF5oBNUOpAQ,wSTuiTk-sKNdcFyprzZAjg,B5XSoSG3SfvQGtKEGQ1tSQ,3.0,1,1,0,This easter instead of going to Lopez Lake we ...,2016-03-30 22:46:33
1,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3.0,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06
2,eCiWBf1CJ0Zdv1uVarEhhw,OhECKhQEexFypOMY6kypRw,vC2qm1y3Au5czBtbhc-DNw,4.0,0,0,0,Yes this is the only sushi place in town. How...,2013-09-04 03:48:20
3,YbMyvlDA2W3Py5lTz8VK-A,4hBhtCSgoxkrFgHa4YAD-w,bbEXAEFr4RYHLlZ-HFssTA,5.0,0,0,0,Great burgers fries and salad! Burgers have a...,2017-01-02 03:17:34
4,L0jv8c2FbpWSlfNC6bbUEA,bFPdtzu11Oi0f92EAcjqmg,IDtLPgUrqorrpqSLdfMhZQ,5.0,0,0,0,What a great addition to the Funk Zone! Grab ...,2016-10-13 22:50:47


In [45]:
X = df_reviews.drop('stars', axis=1)
X

,review_id,user_id,business_id,useful,funny,cool,text,date
0,ZKvDG2sBvHVdF5oBNUOpAQ,wSTuiTk-sKNdcFyprzZAjg,B5XSoSG3SfvQGtKEGQ1tSQ,1,1,0,This easter instead of going to Lopez Lake we ...,2016-03-30 22:46:33
1,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06
2,eCiWBf1CJ0Zdv1uVarEhhw,OhECKhQEexFypOMY6kypRw,vC2qm1y3Au5czBtbhc-DNw,0,0,0,Yes this is the only sushi place in town. How...,2013-09-04 03:48:20
3,YbMyvlDA2W3Py5lTz8VK-A,4hBhtCSgoxkrFgHa4YAD-w,bbEXAEFr4RYHLlZ-HFssTA,0,0,0,Great burgers fries and salad! Burgers have a...,2017-01-02 03:17:34
4,L0jv8c2FbpWSlfNC6bbUEA,bFPdtzu11Oi0f92EAcjqmg,IDtLPgUrqorrpqSLdfMhZQ,0,0,0,What a great addition to the Funk Zone! Grab ...,2016-10-13 22:50:47
...,...,...,...,...,...,...,...,...
348851,4BknKk03Mxwqp6YoZeu1sA,26_8S7BIL6YRw2PcS99Tnw,fiQxkb1pfPpvrr0oYYYBcA,0,0,0,If I could give more than 5 stars I would! I ...,2021-12-06 22:49:57
348852,IHcg19rAtUsd2LlFMJuO0w,njEH2bZZtBtKAKfL23JOOw,Kthnb7PjDSd7FsBp4wqoHA,0,0,0,I have yet to not like a Trader Joe's and this...,2021-12-22 03:58:07
348853,BnIK6q_3C-6WNXY6M17rRA,weNjiH8wxEF0xrMtceGQcg,q2sbzd5Z7Bi8pkmCyr-Umw,1,0,0,Mitchel was very professional timely and rel...,2021-10-05 18:51:55
348854,_USUb1lBWnO4VV58FHL6fw,ik0UUieUX38rPg24heeVgw,6aUb_f9C7a2oOxaIfRGSdA,0,0,1,I ordered a half rack of Pork Ribs (a la carte...,2019-04-29 18:10:03


In [42]:
Y = df_reviews['stars']
Y

0         3.0
1         3.0
2         4.0
3         5.0
4         5.0
         ... 
348851    5.0
348852    5.0
348853    5.0
348854    4.0
348855    4.0
Name: stars, Length: 348856, dtype: float64

In [46]:
X_train = X[:(int)(len(X)*(0.8))]
X_val = X[(int)(len(X)*(0.8)):(int)(len(X)*(0.9))]
X_test = X[(int)(len(X)*(0.9)):]

Y_train = Y[:int(len(Y)*(0.8))]
Y_val = Y[(int)(len(X)*(0.8)):(int)(len(X)*(0.9))]
Y_test = Y[int(len(Y)*(0.9)):]

In [5]:
usersPerBusiness = defaultdict(set)
businessesPerUser = defaultdict(set)
ratingDict = {} # To retrieve a rating for a specific user/item pair

In [6]:
for idx, d in tqdm(df_reviews.iterrows()):
    user,busn = d['user_id'], d['business_id']
    usersPerBusiness[busn].add(user)
    businessesPerUser[user].add(busn)
    ratingDict[(user,busn)] = d['stars']

348856it [00:35, 9802.47it/s] 


In [7]:
userAverages = {}
itemAverages = {}

for u in businessesPerUser:
    rs = [ratingDict[(u,i)] for i in businessesPerUser[u]]
    userAverages[u] = sum(rs) / len(rs)
    
for i in usersPerBusiness:
    rs = [ratingDict[(u,i)] for u in usersPerBusiness[i]]
    itemAverages[i] = sum(rs) / len(rs)

In [8]:
# usersPerBusiness

In [9]:
# businessesPerUser

In [10]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

In [11]:
def Cosine(i1, i2):
    # Between two items
    inter = usersPerBusiness[i1].intersection(usersPerBusiness[i2])
    numer = 0
    denom1 = 0
    denom2 = 0
    for u in inter:
        numer += ratingDict[(u,i1)]*ratingDict[(u,i2)]
    for u in usersPerBusiness[i1]:
        denom1 += ratingDict[(u,i1)]**2
    for u in usersPerBusiness[i2]:
        denom2 += ratingDict[(u,i2)]**2
    denom = math.sqrt(denom1) * math.sqrt(denom2)
    if denom == 0: return 0
    return numer / denom

In [12]:
def Pearson(i1, i2):
    # Between two items
    iBar1 = itemAverages[i1]
    iBar2 = itemAverages[i2]
    inter = usersPerBusiness[i1].intersection(usersPerBusiness[i2])
    numer = 0
    denom1 = 0
    denom2 = 0
    for u in inter:
        numer += (ratingDict[(u,i1)] - iBar1)*(ratingDict[(u,i2)] - iBar2)
    for u in inter: #usersPerItem[i1]:
        denom1 += (ratingDict[(u,i1)] - iBar1)**2
    #for u in usersPerItem[i2]:
        denom2 += (ratingDict[(u,i2)] - iBar2)**2
    denom = math.sqrt(denom1) * math.sqrt(denom2)
    if denom == 0: return 0
    return numer / denom

In [13]:
# def mostSimilar(i, sim_metric):
#     similarities = []
#     users = usersPerBusiness[i]
#     for i2 in usersPerBusiness:
#         if i2 == i: continue
#         if sim_metric == 'Jaccard':
#           sim = Jaccard(users, usersPerBusiness[i2])
#         elif sim_metric == 'Pearson':
#           sim = Pearson(i, i2)
#         similarities.append((sim,i2))
#     similarities.sort(reverse=True)
#     return similarities[:10]

In [14]:
# mostSimilar('Qkg16mN-8QR66gjzg6gMyw', 'Jaccard') # pass business id

In [15]:
reviewsPerUser = defaultdict(list)
reviewsPerBusiness = defaultdict(list)

In [16]:
for idx, d in tqdm(df_reviews.iterrows()):
    user,item = d['user_id'], d['business_id']
    reviewsPerUser[user].append(d)
    reviewsPerBusiness[item].append(d)

348856it [00:26, 13055.15it/s]


In [17]:
# reviewsPerUser

In [18]:
# reviewsPerBusiness

In [19]:
ratingMean = sum([d['stars'] for idx, d in df_reviews.iterrows()]) / len(df_reviews)

In [20]:
ratingMean

3.9600092874997133

In [ ]:
ratingMean2 = sum([d['stars'] for d in X_train]) / len(X_train)

In [21]:
# reviewsPerUser

In [22]:
def predictRating(user,item, sim_metric):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['business_id']
        if i2 == item: continue
        ratings.append(d['stars'])
        if sim_metric == "Jaccard":
          sim = Jaccard(usersPerBusiness[item],usersPerBusiness[i2])
        elif sim_metric == "Cosine":
          sim = Cosine(i, i2)
        elif sim_metric == "Pearson":
          sim = Pearson(i, i2)
        else:
          continue
        similarities.append(sim)
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return sum(weightedRatings) / sum(similarities)
    else:
        # User hasn't rated any similar items
        return ratingMean

In [23]:
u,i = 'OhECKhQEexFypOMY6kypRw',	'vC2qm1y3Au5czBtbhc-DNw'

In [24]:
predictRating(u, i, "Jaccard")

4.499975962776038

In [25]:
predictRating(u, i, "Cosine")

4.7240100460013945

In [26]:
predictRating(u, i, "Pearson")

5.60050644960074

In [27]:
labels = [d['stars'] for idx, d in tqdm(df_reviews.iterrows())]
labels[:5]

348856it [00:21, 16094.94it/s]


[3.0, 3.0, 4.0, 5.0, 5.0]

In [30]:
alwaysPredictMean = [ratingMean for idx, d in tqdm(df_reviews.iterrows())]

348856it [00:15, 21819.44it/s]


In [31]:
mean_squared_error(alwaysPredictMean, labels)

1.999693539941719

In [32]:
jac_Predictions = [predictRating(d['user_id'], d['business_id'], "Jaccard") for idx, d in tqdm(df_reviews.iterrows())]

348856it [04:52, 1193.36it/s]


In [33]:
jac_Predictions[:5]

[3.9600092874997133,
 4.118875857627681,
 4.499975962776038,
 4.011983736241189,
 3.9600092874997133]

In [34]:
mean_squared_error(jac_Predictions, labels)

2.1566944903106458

In [35]:
cos_Predictions = [predictRating(d['user_id'], d['business_id'], "Cosine") for idx, d in tqdm(df_reviews.iterrows())]

348856it [26:06, 222.66it/s]


In [36]:
cos_Predictions[:5]

[3.9600092874997133,
 2.8661167161084204,
 4.7240100460013945,
 4.094605275408924,
 3.9600092874997133]

In [37]:
mean_squared_error(cos_Predictions, labels)

2.2216698406558133

In [38]:
pear_Predictions = [predictRating(d['user_id'], d['business_id'], "Pearson") for idx, d in tqdm(df_reviews.iterrows())]

348856it [02:10, 2682.21it/s]


In [39]:
pear_Predictions[:5]

[3.9600092874997133,
 -0.4634220262522913,
 5.60050644960074,
 3.9600092874997133,
 3.9600092874997133]

In [40]:
mean_squared_error(pear_Predictions, labels)

5.087173598661708e+27